# Ćwiczenia 6. Walidacja Krzyżowa

## PyTorch na następne ćwiczenia.

Proszę zainstalować pakiet [PyTorch](https://pytorch.org/) oraz torchvision na kolejne zajęcia. Jeśli używane, mając swoje środowisko aktywne użyć:

 * GPU: `conda install pytorch torchvision cudatoolkit=9.0 -c pytorch`
 * tylko CPU: `conda install pytorch torchvision cpuonly  -c pytorch`

## Klasyfikacja

Dzisiaj na zajęciach zajmiemy się problemem klasyfikacji. Podobnie do regresji liniowej jest to przykład uczenia nadzorowanego, ale zamiast przewidywać konkretną liczbę dla danej obserwacji, przewidujemy jego przynajeżność do jednej z *k* klas. Na tych zajęciach będziemy rozważać klasyfikacje binarną, czyli uczyć modele odpowiadające funkcji:

$$ f(x) = y, \quad y \in \{0,1\} $$

Poniżej ładowane są dane, do razu już podzielone na dwie części.

In [2]:
import numpy as np
from utils import get_data

X_train, X_test, y_train, y_test = get_data()

## Zadanie 1.1 (0.5 pkt.)

Używając modelu [`SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) z pakietu sklearn uzyskać 100% dokładność (mierzoną miarą [`accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html))na zbiorze treningowym. Państwa zadanie polega na dobraniu parametru `gamma`, dla ułatwienia proszę nie zmieniać pozostałych domyślnych parametrów. Zalecany przedział parametru `gamma` to wartości z przedziału [0, 1] w skali logarytmicznej.

In [3]:
from sklearn.svm import SVC

# your code here
wp = np.logspace(-10,0,num=10)
wp1 = np.linspace(0.00000001,1,num=100000)
j = 0
for i in wp:
    j=i
    svm1 = SVC(gamma = i)
    svm1.fit(X_train, y_train)
    train_acc1 = svm1.score(X_train, y_train)
    if train_acc1 == 1:
        break
        
print(j)



0.005994842503189421


In [4]:
# test
best_gamma = j

svm = SVC(gamma=best_gamma)
svm.fit(X_train, y_train)
train_acc = svm.score(X_train, y_train)

assert train_acc == 1.

## Zadanie 1.2 (0.5 pkt.)
Używając tej samej rodziny modeli znajdź tym razem taką wartość `gamma`, która daje najlepszy wynik na zbiorze **testowym**. Powinieneś(-aś) być w stanie osiągnąć wynik co najmniej `0.95` accuracy. 

In [5]:
from sklearn.svm import SVC

# your code here
wp2 = np.logspace(-10,0,num=10)
wp3 = np.linspace(0.00000001,1,num=10000)
z = 0
for i in wp2:
    z=i
    svm2 = SVC(gamma = i)
    svm2.fit(X_train, y_train)
    train_acc2 = svm2.score(X_test, y_test)
    if train_acc2 >= 0.95:
        break

print(z)
print(train_acc2)
print(wp2)

2.782559402207126e-06
0.951048951048951
[1.00000000e-10 1.29154967e-09 1.66810054e-08 2.15443469e-07
 2.78255940e-06 3.59381366e-05 4.64158883e-04 5.99484250e-03
 7.74263683e-02 1.00000000e+00]


In [6]:
# test
best_gamma = z # ???

svm = SVC(gamma=best_gamma)
svm.fit(X_train, y_train)
test_acc = svm.score(X_test, y_test)

assert test_acc >= 0.95

### Problem.

**W poprzednim zadaniu zakładaliśmy, że podział na zbiór trenujący/testujący jest nam podany z góry, ale co jeśli go nie mamy?**

Możemy oczywiście sami arbitralnie wybrać część datasetu i uznać ją za nasz zbiór testowy, ale to mogą się z tym wiązać dodatkowe problemy: co jeśli wybrany przez nas fragment jest akurat różny od reszty datasetu, lub odwrotnie?

**Rozwiązanie:** Walidacja Krzyżowa.

1. Podziel dataset na zadaną przez parametr `k` liczbę (prawie) równych grup.
2. Dla każdego podziału, zwróć jedną z tych części jako zbiór testowy, a sumę reszty jako zbiór treningowy.
3. Po nauczeniu łącznie `k` modeli, uśrednij ich wyniki na zbiorach testowych i zwróć jako ostateczny wynik.

## Zadanie 2. (2 pkt.)

Państwa zadaniem jest zaimplementowanie walidacji krzyżowej, czyli funkcji, która dla podanego datasetu w postaci macierzy danych `X` i wektora etykiet `y` zwróci listę `k` krotek: 
  
  `(treningowe_dane, treningowe_etykiety, testowe_dane, testowe_etykiety)`
  
podziałów dokonanych przez walidację krzyżową. Następnie należy użyć modelu z poprzedniego zadania dla policzenia dokładności na zbiorze testowym dla walidacji krzyżowej.

Proszę **nie** korzystać z gotowych rozwiązań dostępnych w pakiecie sklearn.


In [7]:
from typing import List, Tuple


def cross_validation(X: np.ndarray, y: np.ndarray, k: int) -> List[Tuple[np.ndarray, np.ndarray, 
                                                                         np.ndarray, np.ndarray]]:
    # your code here
    X_split = list(split(X,k))
    y_split = list(split(y,k))
    treningowe_dane = []
    treningowe_etykiety = []
    testowe_dane = []
    testowe_etykiety = []
    wynik_koncowy = []
    #print(X_split,y_split,X_conc)
    for i in range(k):
        testowe_dane.clear()
        testowe_etykiety.clear()
        
        X_split_copy = X_split.copy()
        y_split_copy = y_split.copy()
        testowe_dane.append(X_split_copy.pop(i))
        testowe_etykiety.append(y_split_copy.pop(i))
        
        treningowe_dane = np.concatenate(X_split_copy)
        treningowe_etykiety = np.concatenate(y_split_copy)
        
        tsd = np.asarray(testowe_dane[0])
        tse = np.asarray(testowe_etykiety[0])
        trd = np.asarray(treningowe_dane)
        tre = np.asarray(treningowe_etykiety)
        Tuple = (trd,tre,tsd,tse)
        wynik_koncowy.append(Tuple)
    
    return wynik_koncowy
        
        
    

In [8]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))


In [9]:
from checker import test_cv

test_cv(cross_validation)

In [26]:
X, y = get_data(split=False)
# test



#svm.fit(X_train, y_train)
#test_acc = svm.score(X_test, y_test)

#assert test_acc >= 0.95
cv_accuracy = 0
k =5
svm2 = SVC(gamma = 0.00000001)
wynik = cross_validation(X,y,k)
for i in range(k):
    svm2.fit(wynik[i][0], wynik[i][1])
    train_acc2 = svm2.score(wynik[i][2], wynik[i][3])
    #print(train_acc2)
    cv_accuracy += train_acc2
print(cv_accuracy/k) #dokładność vc na zbiorze testowym dla stałego parametru gamma = 0.00000001

0.8489830771619313


## Zadanie 3 (1 pkt.)

Mając już lepszą metodę walidacji naszego rozwiązania Państwa zadaniem jest znalezienia najlepszego zestawu hiperparametrów dla modelu z poprzednich zadań, lecz tym razem będziemy szukać również parametru `C`. Parametru C zaleca się szukać w przedziale $(0, + \infty)$ również w skali logarytmicznej.

W zadaniu należy oczywiście skorzystać z zaimplementowanej przez Państwa walidacji krzyżowej. Ponieważ dla dwóch parametrów `C` oraz `gamma` możliwych kombinacji do przetestowania robi są dość sporo dla przetestowania dużych zakresów zalecane są również inne metody przeszukiwania, takie jak:

* przeszukiwanie po kolei z jednym z parametrów ustalonym na stałą.
* przeszukiwanie losowe obu parametrów

Oczywiście jeśli zasoby na to pozwalają można szukać tzw. "grid searchem".

Powinno udać się Państwu wyciągnąć przynajmniej `0.94` accuracy na walidacji krzyżowej.

In [24]:
from sklearn.svm import SVC

X, y = get_data(split=False)
wp2 = np.logspace(-10,0,num=10) #gamma
wp3 = np.logspace(0,10,num=10) #C
cv_accuracy = 0
wynik1 = 0
gamma_wynik = 0
C_wynik = 0
k =5
# your code here
#print(wp2)
#print(wp3)
for i in range (10):
    cv_accuracy = 0
    svm2 = SVC(gamma = wp2[i], C =wp3[i])
    wynik = cross_validation(X,y,k)
    for j in range(k):
        svm2.fit(wynik[j][0], wynik[j][1])
        train_acc2 = svm2.score(wynik[j][2], wynik[j][3])
        #print(train_acc2)
        cv_accuracy += train_acc2
    #print(cv_accuracy/k)
    if cv_accuracy/k >= 0.94:
        wynik1 = cv_accuracy/k
        gamma_wynik = wp2[i]
        C_wynik = wp3[i]
    
print(wynik1) # nasze accuracy
print(gamma_wynik) # gamma
print(C_wynik) # C

0.9455364073901569
2.782559402207126e-06
27825.59402207126


## Zadanie 4. (3 punkty)

Załóżmy, że naszym problemem jest zdecydowanie, która rodzina modeli *SVM* najlepiej radzi sobei z naszym datasetem. Przez rodzinę rozumiemy tutaj modele SVM z różną *funkcją bazwoą* (zwaną często *funkcją jądra*). W pakiecie mamy dostępne kilka możliwości, włącznie z podawaniem swoich własnych, ale w tym zadaniu skupimy się na czterech dostępnych od ręki: `linear`, `poly`, `rbf`, `sigmoid`.

Wiemy jak znaleźć najlepszy zestaw parametrów dla danej rodziny modeli, zrobiliśmy to do tej pory dla domyślnej funkcji bazowej czyli `rbf`. Jak jednak mamy "uczciwie" porównać wyniki modeli pomiędzy sobą? Do tej pory patrzyliśmy na wyniki modelu dla datasetu testowego i to na podstawie tego wyniku wybieraliśmy najlepsze hiperparametry. Jakie mogą być z tym problemy? Overfitting?

Rozwiązanie: jeszcze jedna walidacja krzyżowa!

1. Pierwsza walidacja krzyżowa podzieli nam nasz zbiór na treningowy i testowy. Te testowe zbiory będą naszymi ostatecznymi zbiorami testowymi, na których nie będziemy w żaden sposób się uczyć czy szukać hiperparametrów. 
2. Następnie nasz zbiór treningowy podzielimy ponownie walidacją krzyżową na dwie części: faktyczny treningowy i walidacyjny. Tych dwóch podziałów będziemy używać jak poprzednio do uczenia modelu i testowania hiperparametrów.
3. Po znalezieniu najlepszego zestawu hiperparametrów nauczymy ostatecznie nasz model na sumie zbiorów treningowego i walidacyjnego i sprawdzimy jego dokładność na ostatecznym zbiorze testowym.


**Uwaga**: parametr `C` używany jest dla każdej możliwej funkcji bazowej. Proszę sprawdzić jakie parametry są używane dla jakich funkcji jądra. 
**Hint**: parametry, które mogą państwa interesować to oczywiście `kernel`, oraz `C`, `degree`, `gamma`, `coef0`.

In [64]:
#POROWNANIE SIGMOID I RBF
from sklearn.svm import SVC
import statistics
X, y = get_data(split=False)

#NESTED CROSS VALIDATION KERNEL = 'rbf'                               (treningowe_dane, treningowe_etykiety, testowe_dane, testowe_etykiety)
gamma_wartosci = np.logspace(-10,0,num=10) #gamma
C_wartosci = np.logspace(0,10,num=10)#C
degree_wartosci = np.linspace(0,9,num = 10)#degree

train_acc_list = []
lista_koncowa_dcv = []

k = 5
for j in range(k):
    first_cross_validation = cross_validation(X,y,k)
    for i in range(k-1):
        train_acc_list.clear()
        second_cross_validation = cross_validation(first_cross_validation[i][0],first_cross_validation[i][1],k-1)
        for w in range(10):
            SVM_dcv = SVC(kernel = 'rbf',C = C_wartosci[w],gamma = gamma_wartosci[w],degree = degree_wartosci[w],coef0 = 0 )
            SVM_dcv.fit(second_cross_validation[i][0], second_cross_validation[i][1])
            train_acc2 = SVM_dcv.score(second_cross_validation[i][2], second_cross_validation[i][3])
            train_acc_list.append(train_acc2)
        training_error_min_index = train_acc_list.index(max(train_acc_list))
        best_gamma = gamma_wartosci[training_error_min_index]
        best_C = C_wartosci[training_error_min_index]
    SVM_dcv = SVC(kernel = 'rbf',C = best_C,gamma = best_gamma)
    SVM_dcv.fit(first_cross_validation[j][0],first_cross_validation[j][1])
    lista_koncowa_dcv.append(SVM_dcv.score(first_cross_validation[j][2],first_cross_validation[j][3]))
srednia_koncowa = statistics.mean(lista_koncowa_dcv)
print('kernel = rbf')
print(lista_koncowa_dcv)
print(srednia_koncowa)

kernel = rbf
[0.8771929824561403, 0.956140350877193, 0.9736842105263158, 0.956140350877193, 0.9469026548672567]
0.9420121099208197


In [81]:
#NESTED CROSS VALIDATION KERNEL = 'sigmoid'   
gamma_wartosci = np.logspace(-10,0,num=10) #gamma
C_wartosci = np.logspace(0,10,num=10)#C
degree_wartosci = np.linspace(0,9,num = 10)#degree

train_acc_list = []
lista_koncowa_dcv = []

k = 5
for j in range(k):
    first_cross_validation = cross_validation(X,y,k)
    for i in range(k-1):
        train_acc_list.clear()
        second_cross_validation = cross_validation(first_cross_validation[i][0],first_cross_validation[i][1],k-1)
        for w in range(10):
            SVM_dcv = SVC(kernel = 'sigmoid',C = C_wartosci[w],gamma = gamma_wartosci[w],degree = degree_wartosci[w],coef0 = 0)
            SVM_dcv.fit(second_cross_validation[i][0], second_cross_validation[i][1])
            train_acc2 = SVM_dcv.score(second_cross_validation[i][2], second_cross_validation[i][3])
            train_acc_list.append(train_acc2)
        training_error_min_index = train_acc_list.index(max(train_acc_list))
        best_gamma = gamma_wartosci[training_error_min_index]
        best_C = C_wartosci[training_error_min_index]
        best_degree = degree_wartosci[training_error_min_index]
    SVM_dcv = SVC(kernel = 'sigmoid',C = best_C,gamma = best_gamma,degree = best_degree,coef0 = 0)
    SVM_dcv.fit(first_cross_validation[j][0],first_cross_validation[j][1])
    lista_koncowa_dcv.append(SVM_dcv.score(first_cross_validation[j][2],first_cross_validation[j][3]))
srednia_koncowa = statistics.mean(lista_koncowa_dcv)
print('kernel = sigmoid')
print(lista_koncowa_dcv)
print(srednia_koncowa)

kernel = sigmoid
[0.6491228070175439, 0.7894736842105263, 0.9035087719298246, 0.9385964912280702, 0.911504424778761]
0.8384412358329452
